#Dumby Scenario

###We are inputting our parameters (rate, years, monthly contribution), but these could be incredibly variable. This chart can be motivating, but probably not very reliable or robust. But, later we can run some simulations and find a distribution of possible future value outcomes in the next scenario.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

def calculate_future_value_continuous_compounding(principal, annual_rate, time_years, monthly_payment):
    """
    Calculate the future value of an investment considering continuous compounding
    for both the initial principal and regular monthly contributions.

    Parameters:
    - principal: Initial amount of money invested.
    - annual_rate: Annual interest rate as a decimal.
    - time_years: Time in years for the investment.
    - monthly_payment: Amount contributed monthly to the investment.

    Returns:
    - Future value of the investment.
    """
    # Calculate future value of the initial principal
    future_value_principal = principal * np.exp(annual_rate * time_years)

    # Convert monthly contributions to an annual equivalent
    annual_contribution = monthly_payment * 12

    # Calculate the future value of continuous contributions using integral of continuous growth function from 0 to t years
    future_value_contributions = annual_contribution * ((np.exp(annual_rate * time_years) - 1) / annual_rate)

    #add future value of principal and fv of contributions
    return future_value_principal + future_value_contributions

# Investment Parameters
initial_investment =
interest_rate = 0.10
investment_periods = np.arange(0, 30, 2.5)  # Start from 0 to 20 years, in 2-year intervals
monthly_contribution =
safe_withdrawal_rate = 0.04  # Safe withdrawal rate


# Calculate future values for the specified time periods
future_values = [calculate_future_value_continuous_compounding(initial_investment, interest_rate, period, monthly_contribution) for period in investment_periods]

growth_results = {period: value for period, value in zip(investment_periods, future_values)}

# Calculate safe withdrawal amounts
safe_withdrawal_amounts = [value * safe_withdrawal_rate for value in future_values]


plt.figure(figsize=(12, 8))

# Plot future values and safe withdrawal amounts
plt.plot(investment_periods, future_values, marker='o', linestyle='-', color='#1f77b4', label='Future Value')
plt.plot(investment_periods, safe_withdrawal_amounts, marker='s', linestyle='dotted', color='#2ca02c', label='Safe Withdrawal Amount (USD$)')

# Enhance chart with title, labels, and grid
plt.title('Investment Future Value and Safe Withdrawal Amounts Over Time', fontsize=16)
plt.xlabel('Investment Duration (Years)', fontsize=14)
plt.ylabel('Amount in Dollars USD ($)', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(loc='upper left', fontsize=12)

# Adjust y-axis to show labels with thousand separator
plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))

# Adjusting text labels for clarity
horizontal_shift = 0.15  # Slight horizontal shift for the future value labels to the left
vertical_offset_future_value = max(future_values) * 0.01  # Vertical offset for future value labels
vertical_offset_withdrawal = -0.03 * max(future_values)  # Negative vertical offset for withdrawal labels

for period, (value, withdrawal) in zip(investment_periods, zip(future_values, safe_withdrawal_amounts)):
    # Shift future value labels to the left
    plt.text(period - horizontal_shift, value + vertical_offset_future_value, f"${value:,.0f}", ha='right', va='bottom', color='#1f77b4', fontsize=10)
    # Place withdrawal amount labels below the curve using negative vertical offset
    plt.text(period, withdrawal + vertical_offset_withdrawal, f"${withdrawal:,.0f}", ha='center', va='top', color='#2ca02c', fontsize=10)

plt.tight_layout()
plt.show()



In [ ]:
def model_growth_with_withdrawals(growth_results, annual_rate, withdrawal_rate, start_year, total_years):
    if start_year in growth_results:
        initial_value = growth_results[start_year]
    else:
        initial_value = growth_results[min(growth_results.keys(), key=lambda k: abs(k - start_year))]

    # Initialize dictionaries to store asset values and withdrawals, including the initial withdrawal
    asset_values = {start_year: initial_value}
    withdrawals = {}

    current_value = initial_value
    # Immediately apply a withdrawal at the start of the start_year
    initial_withdrawal = current_value * withdrawal_rate
    current_value -= initial_withdrawal
    withdrawals[start_year] = initial_withdrawal

    # Now iterate through the remaining years, applying growth and withdrawals every 2 years
    for year in np.arange(start_year + 2, start_year + total_years + 1, 2):
        # Apply growth for 2 years first
        current_value *= np.exp(annual_rate * 2)
        # Then calculate and apply the withdrawal at the start of the period (every 2 years)
        withdrawal_amount = current_value * withdrawal_rate
        current_value -= withdrawal_amount #DEDUCT YOUR WITHDRAWAL BEFORE CALCULATING GROWTH FOR THE YEAR.

        # current_value *= np.exp(annual_rate * 2)
        # Store the results
        asset_values[year] = current_value
        withdrawals[year] = withdrawal_amount

    return asset_values, withdrawals


annual_rate = 0.08 # assume a slightly lower rate through retirement safer allocations, etc.
withdrawal_rate = 0.04

# Correct the start_year to align with the interval from the growth phase
start_year = 15  # Adjust this to match an interval in your growth calculations
total_years = 40

# Now, when you call model_growth_with_withdrawals, it starts at a year that's directly aligned with your growth phase intervals
asset_growth, withdrawals = model_growth_with_withdrawals(growth_results, annual_rate, withdrawal_rate, start_year, total_years)



# Assuming asset_growth is correctly calculated using the updated function
plt.figure(figsize=(10, 6))

# Plot asset values with the correct starting point
plt.plot(list(asset_growth.keys()), list(asset_growth.values()), marker='o', linestyle='-', color='#1f77b4', label='Asset Value After Withdrawals')

plt.title('Asset Growth with Annual Withdrawals', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Asset Value ($)', fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend(loc='upper right', fontsize=12)

plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))

# Annotate both asset values and withdrawals, check if the year exists in withdrawals to avoid KeyError
for year, value in asset_growth.items():
    if year in withdrawals:  # Check if the year exists in withdrawals to avoid KeyError
        withdrawal_amount = withdrawals[year]  # Get the withdrawal amount for the year
        # Annotate withdrawal amount below the asset value annotation
        plt.annotate(f"${withdrawal_amount:,.0f}", (year, value), textcoords="offset points", xytext=(-10,-15), ha='center', color='red', fontsize=8)

    # Annotate asset value
    plt.annotate(f"${value:,.0f}", (year, value), textcoords="offset points", xytext=(-10,10), ha='center', fontsize=8)

plt.tight_layout()
plt.show()




#Now we can conduct real science. Find a distribution of possible FV outcomes

###Now we can run many simulations and *try* to model the randomness in our parameters. This still could be off.. we can never foresee the black swans. But, at least it can be more demonstrative of the randomness possible in these parameters we are setting.

## Be ready for a reality check... our dreamy chart from above probably doesn't hold water once we account for randomness!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from tqdm import tqdm  # Import tqdm for progress bar


# Simulation setup
initial_investment =
time_years =
annual_rate_min = -0.18
annual_rate_max = 0.18
monthly_contribution_min = 0
monthly_contribution_max =
num_simulations = 1000


def invert_transform_data(data, target_min, target_max):
    """
    Transform right-skewed data to left-skewed by inverting its distribution within a specified range.

    :param data: Right-skewed data.
    :param target_min: Minimum value of the target range.
    :param target_max: Maximum value of the target range.
    :return: Inverted left-skewed data within the target range.
    """
    # Invert the distribution
    data_min, data_max = np.min(data), np.max(data)
    inverted_data = data_max + data_min - data

    # Scale the inverted data to the target range
    scaled_inverted_data = (inverted_data - np.min(inverted_data)) / np.ptp(inverted_data)
    return scaled_inverted_data * (target_max - target_min) + target_min

def calculate_variable_compounding(principal, annual_rates, monthly_contributions, time_years):
    """
    This function calculates the future value of an investment given a principal amount,
    variable annual interest rates, and variable monthly contributions over a specified number of years.
    """
    current_value = principal
    monthly_values = []

    for year in range(time_years):
        for month in range(12):
            monthly_rate = annual_rates[min(year, len(annual_rates)-1)] / 12
            current_value = current_value * (1 + monthly_rate) + monthly_contributions[year * 12 + month]
            monthly_values.append(current_value)

    return np.array(monthly_values)


def generate_left_skewed_data(size, min_val, max_val):
    """
    Generate left-skewed data for annual rates within a specified range [min_val, max_val].
    This uses a normal distribution and applies transformations to achieve left skewness and fit the specified range.
    """
    # Generate data from a normal distribution
    data = np.random.normal(loc=(max_val + min_val) / 2.0, scale=(max_val - min_val) / 5.0, size=size)

    # Apply transformations to skew the distribution left
    data = np.clip(data, min_val, max_val)  # Ensure data is within the specified range
    power = 2.3  # Adjust the power parameter for increased skewness
    data = max_val - (max_val - min_val) * np.power((max_val - data) / (max_val - min_val), power)

    return data


# Arrays to collect all annual rates and monthly contributions for histogram plotting
all_annual_rates = np.zeros(num_simulations * time_years)
all_monthly_contributions = np.zeros(num_simulations * time_years * 12)

# Array to hold the end-of-year values for each simulation
all_end_of_year_values = np.zeros((num_simulations, time_years))

# Initialize the progress bar
progress_bar = tqdm(total=num_simulations, desc='Simulations Progress', unit='simulations')

for i in range(num_simulations):
    # Generate left-skewed annual rates
    annual_rates = generate_left_skewed_data(time_years, annual_rate_min, annual_rate_max)
    # Generate right-skewed data for  monthly contributions
    mu, sigma = 0.1, 0.65  # These parameters need fine-tuning
    right_skewed_contributions = np.random.lognormal(mu, sigma, time_years * 12)

    # Transform to left-skewed and scale to desired range
    monthly_contributions = invert_transform_data(right_skewed_contributions, monthly_contribution_min, monthly_contribution_max)

    # Store the generated values for later plotting
    all_annual_rates[i * time_years:(i + 1) * time_years] = annual_rates
    all_monthly_contributions[i * time_years * 12:(i + 1) * time_years * 12] = monthly_contributions

    # Perform the compounding calculation
    monthly_values = calculate_variable_compounding(initial_investment, annual_rates, monthly_contributions, time_years)
    for year in range(time_years):
        all_end_of_year_values[i, year] = monthly_values[(year + 1) * 12 - 1]
    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

# Calculate mean values for plotting the growth chart
mean_end_of_year_values = np.mean(all_end_of_year_values, axis=0)
investment_periods = np.arange(0, time_years, 2)
mean_values_at_selected_periods = mean_end_of_year_values[::2]
safe_withdrawal_amounts = mean_values_at_selected_periods * 0.04

# Plot the growth chart
plt.figure(figsize=(12, 8))
plt.plot(investment_periods, mean_values_at_selected_periods, 'o-', label='Mean Future Value')
plt.plot(investment_periods, safe_withdrawal_amounts, 's--', label='Safe Withdrawal Amount (4%)')
for i, txt in enumerate(mean_values_at_selected_periods):
    plt.text(investment_periods[i], mean_values_at_selected_periods[i], f"${txt:,.0f}", fontsize=8, verticalalignment='bottom', horizontalalignment='right')
for i, txt in enumerate(safe_withdrawal_amounts):
    plt.text(investment_periods[i], safe_withdrawal_amounts[i], f"${txt:,.0f}", fontsize=8, verticalalignment='top', horizontalalignment='right')

plt.title('Mean Investment Future Value and Safe Withdrawal Amounts Over Time')
plt.xlabel('Years')
plt.ylabel('Value ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot histograms of the aggregated annual rates and monthly contributions
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.hist(all_annual_rates, bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Annual Rates')
plt.xlabel('Annual Rate')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(all_monthly_contributions, bins=50, color='lightgreen', edgecolor='black')
plt.title('Distribution of Monthly Contributions')
plt.xlabel('Monthly Contribution')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()
